In [185]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.optimize import curve_fit
import os

In [186]:
GLOBAL_FLOAT_PRECISION_NUMPY = np.float64

## Data preparation

### Load data

In [187]:
chirp_direction = 'chirp_up'
data_file_path = r'C:\Users\vaish\dsvv_atomionics\compiled_data\compiled_' + f'{chirp_direction}.pkl'
data_cols = [
		'master_run_number', 'chirp_rate', 'fraction',
		'CA+_mean', 'CA+_std_dev', 'CA+_percentile_0', 'CA+_percentile_10', 'CA+_percentile_20', 
		'CA+_percentile_30', 'CA+_percentile_40', 'CA+_percentile_50', 'CA+_percentile_60', 'CA+_percentile_70', 
		'CA+_percentile_80', 'CA+_percentile_90', 'CA+_percentile_100', 'CA-_mean', 'CA-_std_dev', 'CA-_percentile_0', 
		'CA-_percentile_10', 'CA-_percentile_20', 'CA-_percentile_30', 'CA-_percentile_40', 'CA-_percentile_50', 
		'CA-_percentile_60', 'CA-_percentile_70', 'CA-_percentile_80', 'CA-_percentile_90', 'CA-_percentile_100'
	]

data = pd.DataFrame(pd.read_pickle(data_file_path), columns=data_cols, dtype=GLOBAL_FLOAT_PRECISION_NUMPY)
data = data.astype({'master_run_number': 'int32'})

# 80-20 train/test split
train_data_df = data.sample(frac=0.8, random_state=0)
test_data_df = data.drop(train_data_df.index)

print(train_data_df.shape)
# print(train_data_df.head())
print(test_data_df.shape)
# print(test_data_df.head())

(126, 29)
(32, 29)


In [188]:
compiled_constants_csv_path = r'C:\Users\vaish\dsvv_atomionics\compiled_data\compiled_constants.csv'
compiled_constants_df = pd.read_csv(compiled_constants_csv_path)
compiled_constants_dtypes = {
	'master_run': 'int32',
	'chirp_direction': 'str',
	'bigT': GLOBAL_FLOAT_PRECISION_NUMPY,
	'Keff': GLOBAL_FLOAT_PRECISION_NUMPY,
	'contrast': GLOBAL_FLOAT_PRECISION_NUMPY,
	'g0': GLOBAL_FLOAT_PRECISION_NUMPY,
	'fringe_offset': GLOBAL_FLOAT_PRECISION_NUMPY
}
compiled_constants_df = compiled_constants_df.astype(compiled_constants_dtypes)

print(compiled_constants_df.shape)
# print(compiled_constants_df.head())

(4, 7)


## Custom Loss Function

In [189]:
torch.set_default_dtype(torch.float64)

In [190]:
def sine(alpha, Keff, bigT, contrast, g0, fringe_offset):
    phi = (Keff * g0 - 2 * torch.pi * alpha) * (bigT**2)
    return (-contrast * torch.cos(phi) + fringe_offset)

In [191]:
def get_predicted_fractions_from_outputs(outputs, data_df):
	predicted_fractions = []
	for index in range(outputs.shape[0]):
		data_row = data_df.iloc[index]
		master_run = data_row['master_run_number']
		compiled_constants_row = compiled_constants_df[(compiled_constants_df['master_run'] == master_run) & (compiled_constants_df['chirp_direction'] == chirp_direction)]

		Keff = compiled_constants_row['Keff'].item()
		bigT = compiled_constants_row['bigT'].item()
		contrast = compiled_constants_row['contrast'].item()
		g0 = compiled_constants_row['g0'].item()
		fringe_offset = compiled_constants_row['fringe_offset'].item()
		current_output = outputs[index]

		predicted_fraction = sine(current_output, Keff, bigT, contrast, g0, fringe_offset)
		predicted_fractions.append(predicted_fraction)
	
	predicted_fractions_tensor = torch.stack(predicted_fractions, 0)
	return predicted_fractions_tensor


In [192]:
# A little test you can run, to verify that gradients do propagate through this transformation
# https://stackoverflow.com/questions/70426391/how-to-transform-output-of-nn-while-still-being-able-to-train

# start with some random tensor representing the input predictions
# make sure it requires_grad
pred = torch.rand((4, 5, 2, 3)).requires_grad_(True)
# transform it
tpred = get_predicted_fractions_from_outputs(pred, train_data_df)

# make up some "default" loss function and back-prop
tpred.mean().backward()

# check to see all gradients of the original prediction:
pred.grad

tensor([[[[-3.0785e-07, -3.0772e-07, -3.0782e-07],
          [-3.0783e-07, -3.0781e-07, -3.0771e-07]],

         [[-3.0777e-07, -3.0774e-07, -3.0785e-07],
          [-3.0772e-07, -3.0774e-07, -3.0781e-07]],

         [[-3.0782e-07, -3.0780e-07, -3.0786e-07],
          [-3.0783e-07, -3.0776e-07, -3.0779e-07]],

         [[-3.0781e-07, -3.0777e-07, -3.0772e-07],
          [-3.0783e-07, -3.0779e-07, -3.0775e-07]],

         [[-3.0782e-07, -3.0777e-07, -3.0774e-07],
          [-3.0783e-07, -3.0782e-07, -3.0776e-07]]],


        [[[-3.0773e-07, -3.0773e-07, -3.0774e-07],
          [-3.0772e-07, -3.0783e-07, -3.0783e-07]],

         [[-3.0772e-07, -3.0786e-07, -3.0784e-07],
          [-3.0787e-07, -3.0778e-07, -3.0771e-07]],

         [[-3.0781e-07, -3.0786e-07, -3.0787e-07],
          [-3.0783e-07, -3.0771e-07, -3.0781e-07]],

         [[-3.0783e-07, -3.0785e-07, -3.0780e-07],
          [-3.0770e-07, -3.0780e-07, -3.0776e-07]],

         [[-3.0777e-07, -3.0781e-07, -3.0786e-07],
          [

## Benchmark MSE score

In [193]:
test_chirp_rates = torch.tensor(test_data_df['chirp_rate'].values)
test_fractions = torch.tensor(test_data_df['fraction'].values)

In [194]:
predicted_fractions_tensor = get_predicted_fractions_from_outputs(test_chirp_rates, test_data_df)
residuals_baseline = test_fractions - predicted_fractions_tensor
mse_baseline = torch.mean(residuals_baseline ** 2)
print(f'baseline MSE: {mse_baseline}')

baseline MSE: 0.006329568853331391


## Training

### Neural Network

In [195]:
class TransformationNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(TransformationNN, self).__init__()
        # Define the layers of the network: 2 fully connected layers 27 -> 14 -> 7 -> 1
        self.fc1 = nn.Linear(input_dim, 14)
        self.fc2 = nn.Linear(14, 7)
        self.fc3 = nn.Linear(7, output_dim)
    
    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

In [196]:
training_data_tensor = torch.tensor(train_data_df.drop(columns=['fraction', 'master_run_number']).values)
observed_fractions_tensor = torch.tensor(train_data_df['fraction'].values).view(-1, 1)

In [182]:
# Initialize the neural network
model = TransformationNN(input_dim=27, output_dim=1)
optimizer = optim.Adam(model.parameters(), lr=1)
mse_loss_fn = nn.MSELoss()

# Train the model
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()

    # Forward pass: Compute predicted y by passing x to the model
    outputs = model(training_data_tensor)
    predicted_tensors = get_predicted_fractions_from_outputs(outputs, train_data_df)

    loss = mse_loss_fn(predicted_tensors, observed_fractions_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % (num_epochs / 10) == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], MSE: {loss.item()}')

Epoch [1/1000], MSE: 0.01598096918493165
Epoch [101/1000], MSE: 0.010545176267308099
Epoch [201/1000], MSE: 0.008056009626132128
Epoch [301/1000], MSE: 0.007425348818370654
Epoch [401/1000], MSE: 0.007300608618621926
Epoch [501/1000], MSE: 0.007279224568808775
Epoch [601/1000], MSE: 0.00727613553982301
Epoch [701/1000], MSE: 0.007275778020363267
Epoch [801/1000], MSE: 0.007275746144737557
Epoch [901/1000], MSE: 0.007275744013178325


In [183]:
test_chirp_rates_with_noise_tensor = torch.tensor(test_data_df.drop(columns=['fraction', 'master_run_number']).values, dtype=torch.float64)

# Set model to evaluation mode
model.eval()

# Forward pass through the model
with torch.no_grad():
    corrected_chirp_rates = model(test_chirp_rates_with_noise_tensor)
    predicted_fractions = get_predicted_fractions_from_outputs(corrected_chirp_rates, test_data_df)

test_fractions_shape_corrected = test_fractions.view(-1, 1)

# Calculate residuals and MSE
residuals = predicted_fractions - test_fractions_shape_corrected
mse = torch.mean(residuals ** 2)
print(f'eval MSE: {mse.item()}')


MSE: 0.007288108624484816
